In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

img_array= cv2.imread("train/0/Training_859859.jpg")

img_array.shape

plt.imshow(img_array)

Datadirectory = "train/"

Classes = ["0", "1", "2", "3", "4", "5", "6"]

for category in Classes:
    path = os.path.join(Datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))
#         backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break
    
img_size = 224 #==> 224 x 224
new_array = cv2.resize(img_array, (img_size, img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

new_array.shape

# Read all the images and converting them to array

training_Data= [] # Data Array

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:      
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_training_Data()

print(len(training_Data))

import random

random.shuffle(training_Data)

X = []
y = []

for features, label in training_Data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, img_size, img_size, 3) #Converting it into 4 Dimension

X.shape

from PIL import Image

resized_images = [np.array(Image.fromarray(img).resize((img_size, img_size))) for img in X]

# Normalize The Data
#X = X/255.0;

type(y)

Y = np.array(y)

Y.shape

# Deep Learning Model for Training

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 

model = tf.keras.applications.MobileNetV2() #Pre- Trained model

model.summary()

# Transfer Learning - Training, weightts will start from last check point

base_input = model.layers[0].input
base_output = model.layers[-2].output
base_output

final_output = layers.Dense(128)(base_output) #adding new layer, after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) #activation function
final_output = layers.Dense(64)(final_output) 
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output) #My classes are 7, classification layer

final_output

new_model = keras.Model(inputs = base_input, outputs = final_output)

new_model.summary()

new_model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

new_model.fit(X,Y, epochs = 30)

new_model.save('Emotion_Detection.h5')

new_model = tf.keras.models.load_model('Emotion_Detection.h5')

new_model.evaluate

frame = cv2.imread("happyNegro.jpg")

frame.shape

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

gray.shape

faces = faceCascade.detectMultiScale(gray, 1.1, 4)
for x,y,w,h in faces:
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = frame[y:y+h, x:x+w]
    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2) #BGR
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:
        print("Face not Detected")
    else:
        for (ex, ey, ew, eh) in facess:
            face_roi = roi_color[ey: ey+eh, ex:ex + ew]
            
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

plt.imshow(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))

final_image = cv2.resize(face_roi, (224, 224))
final_image = np.expand_dims(final_image, axis = 0) #need 4th Dimension
final_image = final_image/255.0 #Normalzing

Predictions = new_model.predict(final_image)

Predictions[0]

np.argmax(Predictions)